In [20]:
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
#Reddit Set-up
import praw

user_agent = "mick-app 1.0 by u/Adventurous-Mick"
reddit = praw.Reddit(
    client_id= "hLdss6oBnxKzc7WtofWVbg",
    client_secret= "-wfl9qJAURe78Jblh4qIPghc2ctwgQ",
    user_agent= user_agent
)

In [22]:
#hot new rising sumbmission in Subreddit
headlines = set()
for submissions in reddit.subreddit('politics').hot(limit=10):
    print("Title:",submissions.title)
    print("ID:",submissions.id)
    print("author:",submissions.author)
    print("Created",submissions.created_utc)
    print("Score:",submissions.score)
    print("Upvote ratio:",submissions.upvote_ratio)
    print("URL:",submissions.url)
    print("Comment count:",submissions.num_comments)
    print("\n")

Title: Donald Trump's Israel intel leak under scrutiny after Hamas attack
ID: 173rf6z
author: vincevega87
Created 1696857012.0
Score: 6592
Upvote ratio: 0.96
URL: https://www.newsweek.com/donald-trump-israel-intel-russia-hamas-attack-1833094
Comment count: 416


Title: What Ukrainian Soldiers Really Think of Trump and the GOP - For starters: “Donald Trump is a fucking asshole.”
ID: 173oqw2
author: Picture-unrelated
Created 1696848348.0
Score: 8453
Upvote ratio: 0.92
URL: https://newrepublic.com/article/175997/ukrainian-soldiers-really-think-trump-gop
Comment count: 396


Title: RFK Jr. is expected to drop his Democratic primary bid and launch an independent or third-party run
ID: 173p9t7
author: Drumlin
Created 1696850317.0
Score: 1569
Upvote ratio: 0.93
URL: https://apnews.com/article/rfk-jr-presidential-campaign-independent-2024-30d940109c4956de9c81f332ec418463
Comment count: 312


Title: The US will likely 'go to war' in Israel with air and naval power if Syria or Iran become active

In [23]:
#hot new rising top
headlines = set()
for submissions in reddit.subreddit('politics').hot(limit=None):
    headlines.add(submissions.title)
print(len(headlines))

718


In [24]:
df = pd.DataFrame(headlines)
df.head()

,0
0,‘They didn’t stand up to Trump’: how the Repub...
1,Opinion: Matt Gaetz broke Kevin McCarthy for s...
2,'Squad' Dems face backlash calling for 'ceasef...
3,Why Rule 26 is trending amid talk of Trump as ...
4,‘We Put Sharp Knives in the Hands of Children’


In [25]:
df.to_csv('headlines.csv',header=False, encoding='utf-8', index=False)

In [26]:
#Downloading dataset for sentiment analyser
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to C:\Users\ABINAYA
[nltk_data]     THULSI\AppData\Roaming\nltk_data...


True

In [32]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
results= []

for line in headlines:
    pol_score = sia.polarity_scores(line) 
    pol_score['headline'] = line
    results.append(pol_score)
pprint(results[:3],width=100)

[{'compound': 0.4019,
  'headline': '‘They didn’t stand up to Trump’: how the Republican party descended into disarray',
  'neg': 0.0,
  'neu': 0.816,
  'pos': 0.184},
 {'compound': -0.4215,
  'headline': 'Opinion: Matt Gaetz broke Kevin McCarthy for sport',
  'neg': 0.286,
  'neu': 0.714,
  'pos': 0.0},
 {'compound': -0.8779,
  'headline': "'Squad' Dems face backlash calling for 'ceasefire' after Israel attacks: 'Can’t "
              "merely condemn terrorism'",
  'neg': 0.479,
  'neu': 0.521,
  'pos': 0.0}]


In [34]:
df=pd.DataFrame.from_records(results)
df.head()

,neg,neu,pos,compound,headline
0,0.000,0.816,0.184,0.4019,‘They didn’t stand up to Trump’: how the Repub...
1,0.286,0.714,0.000,-0.4215,Opinion: Matt Gaetz broke Kevin McCarthy for s...
2,0.479,0.521,0.000,-0.8779,'Squad' Dems face backlash calling for 'ceasef...
3,0.000,1.000,0.000,0.0000,Why Rule 26 is trending amid talk of Trump as ...
4,0.000,1.000,0.000,0.0000,‘We Put Sharp Knives in the Hands of Children’


In [35]:
df['label'] = 0
df.loc[df['compound'] > 0.2, 'label'] = 1
df.loc[df['compound'] < -0.2, 'label'] = -1
df.head()

,neg,neu,pos,compound,headline,label
0,0.000,0.816,0.184,0.4019,‘They didn’t stand up to Trump’: how the Repub...,1
1,0.286,0.714,0.000,-0.4215,Opinion: Matt Gaetz broke Kevin McCarthy for s...,-1
2,0.479,0.521,0.000,-0.8779,'Squad' Dems face backlash calling for 'ceasef...,-1
3,0.000,1.000,0.000,0.0000,Why Rule 26 is trending amid talk of Trump as ...,0
4,0.000,1.000,0.000,0.0000,‘We Put Sharp Knives in the Hands of Children’,0


In [36]:
df2 = df[['headline','label']]

In [37]:
df2.to_csv('reddit_headlines_label.csv',header=False, encoding='utf-8', index=False)

In [40]:
df.label.value_counts()

 0    302
-1    292
 1    124
Name: label, dtype: int64

In [38]:
df.label.value_counts(normalize=True) * 100

 0    42.061281
-1    40.668524
 1    17.270195
Name: label, dtype: float64

In [39]:
print("Positive headlines:\n")
pprint(list(df[df['label'] == 1].headline)[:5], width=200)

print("\nNegative headlines:\n")
pprint(list(df[df['label'] == -1].headline)[:5], width=200)

Positive headlines:

['‘They didn’t stand up to Trump’: how the Republican party descended into disarray',
 'The Supreme Court Eyes Its Next Big Power Grab: Two cases this term might cement a new order of judicial supremacy—and end the idea of a government by "we, the people."',
 'Trump in new donor push claims a Biden win means ‘this country will die’',
 "Supreme Court is increasingly putting Christians' First Amendment rights ahead of others' dignity and rights to equal protection",
 'US public support declines for arming Ukraine, Reuters/Ipsos poll shows']

Negative headlines:

['Opinion: Matt Gaetz broke Kevin McCarthy for sport',
 "'Squad' Dems face backlash calling for 'ceasefire' after Israel attacks: 'Can’t merely condemn terrorism'",
 'Amazon’s Alexa has been claiming the 2020 election was stolen',
 "The Republican field is blaming Joe Biden for dealing with Iran after Hamas' attack on Israel",
 'California Becomes First State to Ban Four Harmful Chemicals in Food']


In [6]:
import requests

user_agent = "mick-app 1.0 by u/Adventurous-Mick"
client_id= "hLdss6oBnxKzc7WtofWVbg",
client_secret= "-wfl9qJAURe78Jblh4qIPghc2ctwgQ",
    
# Make a simple Reddit API request (in this case, requesting the Reddit front page)
url = 'https://www.reddit.com/.json'

# Set headers including your user agent
headers = {
    'User-Agent': user_agent,
}

# Use client credentials to authenticate (if needed)
auth = requests.auth.HTTPBasicAuth(client_id, client_secret)

# Make the API request
response = requests.get(url, headers=headers, auth=auth)

# Check the response headers for rate limit information
remaining_requests = response.headers.get('x-ratelimit-remaining')
reset_time = response.headers.get('x-ratelimit-reset')

print(f'Remaining requests: {remaining_requests}')
print(f'Rate limit reset time: {reset_time}')

Remaining requests: 95
Rate limit reset time: 37


In [ ]:
https://www.youtube.com/watch?v=8VZhog5C3bU&ab_channel=PatrickLoeber